In [ ]:
using Plots
using LinearAlgebra
using LaTeXStrings
using Lux, MLUtils, Optimisers, Zygote, OneHotArrays, Random, Statistics, Printf, Reactant
#using MLDatasets: MNIST
using SimpleChains: SimpleChains
using JLD2



In [2]:
particles = 3 # number of qubits
d = 2^particles # dimension of the Hilbert space

# Matrices de Pauli, será nuestra base. 
sigmax = [0 1; 1 0]
sigmay = [0 -im; im 0]
sigmaz = [1 0; 0 -1]
id= [1 0; 0 1] # Matriz identidad

Sigma = Dict(0 => id, 1 => sigmax, 2 => sigmay, 3 => sigmaz) #Diccionario para llamar a las matrices de Pauli
     

Dict{Int64, Matrix} with 4 entries:
  0 => [1 0; 0 1]
  2 => Complex{Int64}[0+0im 0-1im; 0+1im 0+0im]
  3 => [1 0; 0 -1]
  1 => [0 1; 1 0]

In [3]:
k = 2 # Número de operadores locales que queremos generar

list=[0:3 for _ in 1:k] # Arreglo de tres secuencias de 0 a 3

2-element Vector{UnitRange{Int64}}:
 0:3
 0:3

In [4]:
# Generamos una lista de combinaciones de los operador local A1 
place = 1 # Se utiliza para considerar la interacción entre el primer y segundo qubit (Los elementos de A1) 
test_visual_list = []

for i in Base.product(list...)
    total_list=zeros(particles)
    total_list[place:place+k-1]= i|>collect
    if sum(total_list) != 0
        ##print(total_list)
        push!(test_visual_list,total_list)
    end
   # print("\n")
end

ArrayA1 = test_visual_list

15-element Vector{Any}:
 [1.0, 0.0, 0.0]
 [2.0, 0.0, 0.0]
 [3.0, 0.0, 0.0]
 [0.0, 1.0, 0.0]
 [1.0, 1.0, 0.0]
 [2.0, 1.0, 0.0]
 [3.0, 1.0, 0.0]
 [0.0, 2.0, 0.0]
 [1.0, 2.0, 0.0]
 [2.0, 2.0, 0.0]
 [3.0, 2.0, 0.0]
 [0.0, 3.0, 0.0]
 [1.0, 3.0, 0.0]
 [2.0, 3.0, 0.0]
 [3.0, 3.0, 0.0]

In [5]:
place = 2 # Se utiliza para considerar la interacción entre el segundo y tercero qubit (Los elementos de A2)

test_visual_list = [] # Se tiene que volver a definir esta lista como una lista vacía 

for i in Base.product(list...)
    total_list=zeros(particles)
    total_list[place:place+k-1]= i|>collect
    if sum(total_list) != 0
        ##print(total_list)
        push!(test_visual_list,total_list)
    end
   # print("\n")
end

ArrayA2 = test_visual_list

15-element Vector{Any}:
 [0.0, 1.0, 0.0]
 [0.0, 2.0, 0.0]
 [0.0, 3.0, 0.0]
 [0.0, 0.0, 1.0]
 [0.0, 1.0, 1.0]
 [0.0, 2.0, 1.0]
 [0.0, 3.0, 1.0]
 [0.0, 0.0, 2.0]
 [0.0, 1.0, 2.0]
 [0.0, 2.0, 2.0]
 [0.0, 3.0, 2.0]
 [0.0, 0.0, 3.0]
 [0.0, 1.0, 3.0]
 [0.0, 2.0, 3.0]
 [0.0, 3.0, 3.0]

In [6]:
# Valores aleatorios. (1000 datos)
c = rand(2,1000)*2 .-1 # Estos son los valores de y, los que queremos predecir, cada columna es un valor de y (contiene dos elementos)

xaleatMat1 = rand(2,length(ArrayA1)).*2 .-1 

A1P = zeros(2^particles, 2^particles)  # Un arreglo vacío para almacenar matrices
A2P = zeros(2^particles, 2^particles)  # Un arreglo vacío para almacenar matrices

for k in 1:length(ArrayA1)
    A1P += xaleatMat1[1,k] * kron([Sigma[ArrayA1[k][j]] for j in eachindex(ArrayA1[1])]...)
end

for k in 1:length(ArrayA2)
    A2P += xaleatMat1[2,k]*kron([Sigma[ArrayA2[k][i]] for i in eachindex(ArrayA2[1])]...)
end

In [7]:
# En esta celda lo que haremos es generar un vector de tuplas con la información (x,y) donde x es un vector   
# que contiene los promedios a1 y a2 con el estado 0,1,2,... después el vector y tendrá los valores c1 y c2 
# estos valores se repetirán para todos los estados y promedios que consideramos del mismo Hamiltoniano, 
# es decir, si tenemos 4 estados y 2 promedio por estado, se repetirán 4 veces lso valores de y: c1 y c2
# Tomamos para x la mitad de los estados del sistema

n = 1000 # Número de datos que queremos generar 

# Inicializamos un vector vacío para almacenar las 1000 tuplas 
Tup = []

for k in 1 : n

    mat1a1 = zeros(2) # Colocamos un vector de ceros que se va a reiniciar en cada paso 

    # Formamos el Hamiltoniano
    H12 = c[1,k]*A1P + c[2,k]*A2P

    # Tomamos los vectores propios 

    vor  = eigen(H12).vectors
    adjvor = adjoint(vor)

    vor[:,1] # Este es el primer vector propio, los vectores propios con eigen se colocan en columnas 
    adjvor[1,:] # Este es el primer vector propio para la adjunta (Recordemos es un vector izquierdo [Fila])

    # Calculamos el valor esperado de la primera mitad de vectores propios

    ExpValA = zeros(ComplexF64, Int((size(vor)[1])/2))
    for i in 1:Int((size(vor)[1])/2)
        ExpValA[i] = transpose(adjvor[i,:])*A1P*vor[:,i]
    end

    ExpValB = zeros(ComplexF64,Int((size(vor)[1])/2))  # ExpVal2[1] = a21 = \bra{\phi1_}A_2\ket{\phi_1}
    for i in 1:Int((size(vor)[1])/2)
        ExpValB[i] = transpose(adjvor[i,:])*A2P*vor[:,i]
    end



    #tk = (mat1a1, c[:,k]) # Generamos una tupla con un vector que tiene 2 promedios y un vector con c1 y c2 para el respectivo Hamiltoniano
    #push!(Tup,tk) # Agregar la tupla al vector 

    for i in 1:4
        tk = ([real(ExpValA[i]), real(ExpValB[i])], c[:, k])
        push!(Tup, tk)
    end
   # ValsCompleto = vcat(ValsA, ValsB) # Por último nuestro vector tendrá la forma de 8x1, para poder agregarlo al batch 
end 

In [8]:
function loadata(batchsize, train_split)
    # Separar la tupla anterior en los dos batches de datos, los elementos de x y los de y. 

    # Extract inputs (x) and outputs (y) from the tuples
    x_batch = reduce(hcat, getindex.(Tup, 1)) #cat(map(t -> t[1], Tup)...; dims=2) 
    y_batch = reduce(hcat, getindex.(Tup, 2))#cat(map(t -> t[2], Tup)...; dims=2)  # Stack outputs along the 2nd dimension (batch)
    (x_train, y_train), (x_test, y_test) = splitobs((x_batch, y_batch); at=train_split) # Separa los datos en train y test

    return (
        # Use DataLoader to automatically minibatch and shuffle the data
        DataLoader(collect.((x_train, y_train)); batchsize, shuffle=true, partial=false),
        # Don't shuffle the test data
        DataLoader(collect.((x_test, y_test)); batchsize, shuffle=false, partial=false),
    )
end

# batchsize será el tamaño del subconjunto de datos
# train_split, en decimal, el porcentaje de datos que se utilizarán para entrenamiento, el complemento será para test. 

loadata (generic function with 1 method)

In [9]:
myleakyrelu(x) = leakyrelu(x,0.1)

model = Chain(
    Dense(2,64),
    Dense(64,32,myleakyrelu),
    Dense(32,2)
)

Chain(
    layer_1 = Dense(2 => 64),           # 192 parameters
    layer_2 = Dense(64 => 32, myleakyrelu),  # 2_080 parameters
    layer_3 = Dense(32 => 2),           # 66 parameters
)         # Total: 2_338 parameters,
          #        plus 0 states.

In [10]:
# Función de pérdida

CosineSimilarity(x,y) = sum(x.*y)/sqrt(sum(x.^2)*sum(y.^2)) # x, y no son los mismos que los del modelo
function myloss(x,y)
    return 1 - CosineSimilarity(x,y)
end

function loss_fn(model,ps,st,d)
    x,y = d
    ŷ,stn = model(x,ps,st)

    return myloss(ŷ,y),stn,(;)
end

loss_fn (generic function with 1 method)

In [11]:
adaptor = ToSimpleChainsAdaptor((2,))  # forma de entrada
simple_chains_model = adaptor(model)
     

SimpleChainsLayer(
    Chain(
        layer_1 = Dense(2 => 64),       # 192 parameters
        layer_2 = Dense(64 => 32, myleakyrelu),  # 2_080 parameters
        layer_3 = Dense(32 => 2),       # 66 parameters
    ),
)         # Total: 2_338 parameters,
          #        plus 0 states.

In [12]:
const lossfn = loss_fn
# const lossfn = CrossEntropyLoss(; logits=Val(true)) # const se usa para decirle a Julia que el tipo de la variable global no va a cambiar: https://docs.julialang.org/en/v1/base/base/#const
function accuracy(model, ps, st, dataloader)
    total_correct, total = 0, 0
    st = Lux.testmode(st) # Se configura el modelo para hacer inferencia.
    for (x, y) in dataloader
        target_class = onecold(Array(y)) # Es tomar el elemento con mayor probabilidad. Equivalentemente es softmax con temperature cero.
        predicted_class = onecold(Array(first(model(x, ps, st)))) # Es tomar el elemento con mayor probabilidad. Equivalentemente es softmax con temperature cero.
        total_correct += sum(target_class .== predicted_class)
        total += length(target_class)
    end
    return total_correct / total
end
     

accuracy (generic function with 1 method)

In [13]:
fieldnames(Adam) 

(:eta, :beta, :epsilon)

In [19]:
function train(model, dev=cpu_device(); rng=Random.default_rng(), kwargs...)
    train_dataloader, test_dataloader = dev(loadata(100, 0.9))
    ps, st = dev(Lux.setup(rng, model)) # se inicializan los parámetros del modelo de forma aleatoria y se cargan en el CPU (dev)

    vjp = dev isa ReactantDevice ? AutoEnzyme() : AutoZygote() # Usando Reactant permite compilar el modelo antes de entrenarlo: https://lux.csail.mit.edu/stable/manual/compiling_lux_models#reactant-compilation

    train_state = Training.TrainState(model, ps, st, Adam(3.0f-4)) 

    if dev isa ReactantDevice
        x_ra = first(test_dataloader)[1]
        model_compiled = @compile model(x_ra, ps, Lux.testmode(st)) # Justo aquí es compilado el modelo
    else
        model_compiled = model
    end

    ### Lets train the model
    nepochs = 10 # Cuantas veces se pasa por todos los datos
    tr_acc, te_acc = 0.0, 0.0 # Se inicializan las variables de accuracy
    for epoch in 1:nepochs
        stime = time()
        for (x, y) in train_dataloader # Dos preguntas: x y y ya son arreglos o son otro objeto? y en cada evaluación se reordenan?
            _, _, _, train_state = Training.single_train_step!(
                vjp, lossfn, (x, y), train_state
            )
        end
        ttime = time() - stime

        tr_acc =
            accuracy(
                model_compiled, train_state.parameters, train_state.states, train_dataloader
            ) * 100
        te_acc =
            accuracy(
                model_compiled, train_state.parameters, train_state.states, test_dataloader
            ) * 100

        @printf "[%2d/%2d] \t Time %.2fs \t Training Accuracy: %.2f%% \t Test Accuracy: \
                 %.2f%%\n" epoch nepochs ttime tr_acc te_acc
    end

    return train_state.parameters, train_state.states, tr_acc, te_acc # En el código del tutorial no están las primeras dos variables, lo modifiqué para que nos devuelva los parámetros entrenados.
end
     

LoadError: LoadError: UndefVarError: `@compile` not defined in `Main`
Suggestion: check for spelling errors or missing imports.
in expression starting at d:\Git\MCSN\MCSN_OUT\Proyecto\jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X16sZmlsZQ==.jl:11

In [21]:
function train(model, dev=cpu_device(); rng=Random.default_rng(), kwargs...)
    train_dataloader, test_dataloader = dev(loadata(100, 0.9))
    ps, st = dev(Lux.setup(rng, model)) # inicializa los parámetros y estados en el dispositivo dev

    # Siempre se usará AutoZygote en lugar de AutoEnzyme (porque no se usa Reactant)
    vjp = AutoZygote()

    train_state = Training.TrainState(model, ps, st, Adam(3.0f-4))
    
    # Se asigna el modelo tal cual, sin compilación especial
    model_compiled = model

    ### Bucle de entrenamiento
    nepochs = 10 # Cuántas veces se pasa por todos los datos
    tr_acc, te_acc = 0.0, 0.0
    for epoch in 1:nepochs
        stime = time()
        for (x, y) in train_dataloader
            _, _, _, train_state = Training.single_train_step!(
                vjp, lossfn, (x, y), train_state
            )
        end
        ttime = time() - stime

        tr_acc = accuracy(
            model_compiled, train_state.parameters, train_state.states, train_dataloader
        ) * 100
        te_acc = accuracy(
            model_compiled, train_state.parameters, train_state.states, test_dataloader
        ) * 100

        @printf "[%2d/%2d] \t Time %.2fs \t Training Accuracy: %.2f%% \t Test Accuracy: %.2f%%\n" epoch nepochs ttime tr_acc te_acc
    end

    return train_state.parameters, train_state.states, tr_acc, te_acc
end


train (generic function with 2 methods)

In [22]:
tr_acc, te_acc = train(model, reactant_device()); # entrenando el modelo en lux

┌ Warning: `ReactantDevice` is not loaded or not functional. Load `Reactant.jl` before calling this function. Defaulting to CPU.
└ @ MLDataDevices C:\Users\52331\.julia\packages\MLDataDevices\gyWcF\src\public.jl:247


[ 1/10] 	 Time 33.27s 	 Training Accuracy: 64.14% 	 Test Accuracy: 60.50%
[ 2/10] 	 Time 0.00s 	 Training Accuracy: 84.11% 	 Test Accuracy: 80.00%
[ 3/10] 	 Time 0.02s 	 Training Accuracy: 86.58% 	 Test Accuracy: 86.00%
[ 4/10] 	 Time 0.01s 	 Training Accuracy: 83.67% 	 Test Accuracy: 84.25%
[ 5/10] 	 Time 0.04s 	 Training Accuracy: 84.44% 	 Test Accuracy: 85.25%
[ 6/10] 	 Time 0.02s 	 Training Accuracy: 85.53% 	 Test Accuracy: 86.75%
[ 7/10] 	 Time 0.03s 	 Training Accuracy: 86.39% 	 Test Accuracy: 87.75%
[ 8/10] 	 Time 0.02s 	 Training Accuracy: 87.44% 	 Test Accuracy: 88.25%
[ 9/10] 	 Time 0.01s 	 Training Accuracy: 87.83% 	 Test Accuracy: 88.50%
[10/10] 	 Time 0.01s 	 Training Accuracy: 88.06% 	 Test Accuracy: 88.25%


In [23]:

ps, st, tr_acc, te_acc = train(simple_chains_model); #entrenando el modelo de simple simple_chains_model

[ 1/10] 	 Time 50.97s 	 Training Accuracy: 84.31% 	 Test Accuracy: 82.00%
[ 2/10] 	 Time 0.00s 	 Training Accuracy: 83.81% 	 Test Accuracy: 83.00%
[ 3/10] 	 Time 0.00s 	 Training Accuracy: 84.53% 	 Test Accuracy: 84.00%
[ 4/10] 	 Time 0.04s 	 Training Accuracy: 84.75% 	 Test Accuracy: 84.25%
[ 5/10] 	 Time 0.00s 	 Training Accuracy: 85.31% 	 Test Accuracy: 85.00%
[ 6/10] 	 Time 0.01s 	 Training Accuracy: 85.94% 	 Test Accuracy: 85.25%
[ 7/10] 	 Time 0.00s 	 Training Accuracy: 86.97% 	 Test Accuracy: 88.00%
[ 8/10] 	 Time 0.02s 	 Training Accuracy: 87.78% 	 Test Accuracy: 88.25%
[ 9/10] 	 Time 0.00s 	 Training Accuracy: 88.25% 	 Test Accuracy: 88.25%
[10/10] 	 Time 0.02s 	 Training Accuracy: 88.64% 	 Test Accuracy: 89.00%
